<a href="https://colab.research.google.com/github/rootwarp/Ethereal/blob/main/colab/TheGraph_Subgraph_Curation_Pilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TheGraph Subgraph Profit Analysis


In [ ]:
import json

import requests
import numpy as np
import pandas as pd

In [ ]:
TOKEN = "83bbce83edd5d53f6398eb5064933c01" #@param {type:"string"}
SUBGRAPH_ID = "8yHBZUvXcKkZnZM7SDSgcRMtbtNwgUQfM37cA37h7cet"

url = f"https://gateway.thegraph.com/api/{ TOKEN }/subgraphs/id/{ SUBGRAPH_ID }"

query = """
{
  allocations(first: 1000, where: {status: Claimed, closedAtEpoch_gt: 830}) {
    indexingRewards
    queryFeesCollected
    queryFeeRebates
    indexingIndexerRewards
    status
    subgraphDeployment {
      id
      originalName
      signalAmount
      signalledTokens
    }
    id
    indexer {
      id
      indexingRewardCut
      indexingRewardEffectiveCut
      queryFeeCut
      queryFeeEffectiveCut
    }
    closedAtEpoch
    createdAtEpoch
    allocatedTokens
  }
}
"""

resp = requests.post(url, json={ "query": query })
resp_data = json.loads(resp.content)

with open("./data.json", "a") as f:
    f.write(json.dumps(resp_data, indent="  "))

# Manipulate the original data


In [ ]:
raw_allocations = resp_data["data"]["allocations"]

allocations = []

for alloc in raw_allocations:
    duration = alloc["closedAtEpoch"] - alloc["createdAtEpoch"]
    indexing_rewards = int(alloc["indexingRewards"]) / 1E18
    query_fees_collected = int(alloc["queryFeesCollected"]) / 1E18
    query_fee_rebates = int(alloc["queryFeeRebates"]) / 1E18
    allocated_tokens = int(alloc["allocatedTokens"]) / 1E18

    performance = (indexing_rewards + query_fee_rebates) / duration
    performance /= allocated_tokens

    allocations.append([
        alloc["id"],
        alloc["subgraphDeployment"]["id"],
        alloc["createdAtEpoch"],
        alloc["closedAtEpoch"],
        allocated_tokens,
        duration,
        indexing_rewards,
        query_fees_collected,
        query_fee_rebates,
        performance,
    ])

columns = ["AllocationID",
           "SubgraphDeploymentID",
           "CreatedEpoch",
           "ClosedEpoch",
           "Allocations",
           "Duration",
           "IndexerReward",
           "QueryFeesCollected",
           "QueryFeeRebate",
           "Performance",
           ]
           
allocation_df = pd.DataFrame(allocations, columns=columns)

sorted_df = allocation_df.sort_values(by="Performance", ascending=False)
sorted_df

,AllocationID,SubgraphDeploymentID,CreatedEpoch,ClosedEpoch,Allocations,Duration,IndexerReward,QueryFeesCollected,QueryFeeRebate,Performance
35,0xfd76f00f907b3c2eda8c4ebbe144fd738a43cd5a,0xc6367b4361db81cab1b3f00895ba78747b52b75db311...,807,835,5000.00,28,122.331090,0.014125,0.006855,0.000874
16,0x9495ece3b7c7d56449b890a93898a6a5e76f75e7,0x89120082f87340211c2625500290d4d8b0e921241195...,811,838,43695.00,27,336.030388,1780.744066,450.618519,0.000667
0,0x0088ba34dea75d6f946c9aa5a7ca3f2d8671e351,0x8e541c695f2c8ee4a1819901ddca3559aeef7ae16c3c...,813,839,500000.00,26,5638.578823,774.539358,296.531456,0.000457
22,0xb0729c5842847b26484b19a3db4a4128b79a382c,0xb22ad52fdf178df22952754a46dd528fd1b7d62608c3...,813,834,600000.00,21,5669.383382,106.318815,42.930752,0.000453
12,0x878288e2837a46928c2636067ca1553a3b1ad1e8,0xab59a037137e1e8dcb769904b557bd00e7bf9d5b8c7f...,838,839,1307840.98,1,567.176892,0.000000,0.000000,0.000434
24,0xb9e23764b885e987db5bd58de605f21edc365f85,0x9e48c3dbcf0974e8626add263f076a017be27d8aa43b...,810,834,513787.00,24,4764.165698,161.798602,59.024558,0.000391
10,0x7e1208c64625d8e13376169e3f0d0dbfaec1d7e9,0xcc51ec0208008adb7186797bcf6e03b1d9d7f81f8048...,823,842,450000.00,19,2914.918783,789.418694,234.760129,0.000368
28,0xd70a635c77d17fb3d0cfd11d4652dc03e3046bf1,0x6cf19c2db610b7c6097149bf2f931432e9fcb779370d...,813,839,300000.00,26,2786.591936,72.240004,42.436635,0.000363
1,0x0556566d62111a3dccaeef33b0602b6f836872b1,0x96da4090db1aa55838567592e53cbea079bd990bc2f2...,823,840,600000.00,17,3612.427123,146.137002,40.312561,0.000358
6,0x2da79c27ad9261eecc269b685d0edc079c9b67ee,0x94b9f00db03c1aa27a063b45084c2c5f69709978b371...,823,840,657269.00,17,3769.320736,595.879986,121.493614,0.000348


# Check the validity of the data


# TODOs
- Check data validity.
- Get subgraphID and match.
- more?